In [1]:
"""
Main script for FCNT tracker. 
"""

# Import custom class and functions
from inputproducer import InputProducer
from tracker import TrackerVanilla
from vgg16 import Vgg16
from sgnet import GNet, SNet
from utils import img_with_bbox, IOU_eval, select_fms

import numpy as np 
import tensorflow as tf
import matplotlib.pylab as plt

from scipy.misc import imresize
from subprocess import call
import sys
import os
import time

tf.app.flags.DEFINE_integer('iter_epoch_sg', 5,
                          """Number of epoches for trainning"""
                          """SGnet works""")
tf.app.flags.DEFINE_integer('batch_size', 35,
                          """Batch size for SGNet trainning"""
                          """SGnet works""")
tf.app.flags.DEFINE_integer('n_samples_per_batch', 5000,
                          """Number of samples per batch for trainning"""
                          """SGnet works""")
tf.app.flags.DEFINE_integer('iter_max', 1349,
							"""Max iter times through imgs""")
tf.app.flags.DEFINE_integer('sel_num', 354,
                          """Number of feature maps selected.""")
tf.app.flags.DEFINE_string('model_name', 'contour_Suv',\
						"""true for train, false for eval""")
FLAGS = tf.app.flags.FLAGS

## Define varies pathes
DATA_ROOT = 'data/Suv'
PRE_ROOT = os.path.join(DATA_ROOT, 'img_loc')
IMG_PATH = os.path.join(DATA_ROOT, 'img')
GT_PATH = os.path.join(DATA_ROOT, 'groundtruth_rect.txt')
VGG_WEIGHTS_PATH = 'vgg16_weights.npz'

if not os.path.isdir(PRE_ROOT):
    os.mkdir(PRE_ROOT)


TB_SUMMARY = os.path.join('tb_summary', FLAGS.model_name)
if not os.path.isdir('tb_summary'):
    os.mkdir('tb_summary')
if not os.path.isdir(TB_SUMMARY):
    os.mkdir(TB_SUMMARY)

CKPT_PATH = 'checkpoint'
if not os.path.isdir(CKPT_PATH):
    os.mkdir(CKPT_PATH)

model_name = FLAGS.model_name+'.ckpt'
CKPT_MODEL = os.path.join(CKPT_PATH, model_name)


def init_vgg(roi_t0, predict=False):
    """
    Initialize a tf.Session and a vgg16 graph. Followed
    by forwarding the vgg net once to predict top5 class labels
    for image generated in the first frame.

    Args:
        roi_t0: np.ndarray with shape (28x28x3), extracted roi in the first frame.
    Returns:
        sess: tf.Session object.
        vgg: Vgg16 class instance.
    """
    print('Classify it with a pre-trained Vgg16 model.')
    t_start = time.time()
    sess = tf.Session()
    sess.run(tf.initialize_all_variables())
    vgg = Vgg16(VGG_WEIGHTS_PATH, sess)
    if predict:
        vgg.print_prob(roi_t0, sess)
    print('Forwarding the vgg net cost : %.2f s'%(time.time() - t_start))
    return sess, vgg

def gen_sel_maps(sess, roi, vgg, idx_c4, idx_c5):
    """Returns selected c4 and c5 maps"""
    if len(roi.shape) == 3: roi = [roi]
    fd = {vgg.imgs : roi}
    c4_arr, c5_arr = sess.run([vgg.conv4_3_norm, vgg.conv5_3_norm], feed_dict=fd)
    c4_maps = c4_arr[...,idx_c4]
    c5_maps = c5_arr[...,idx_c5]
    return c4_maps, c5_maps


def train_SGNets(sess, img, gt, vgg, snet, gnet, inputProducer, idx_c4, idx_c5):
    """
    Train SGnets' variables by minimizing a composite L2 regression losses.

    Args:
        sess: tf.Session object.
        vgg: Vgg16 class instance.
        snet: SNet class instance.
        gnet:  GNet class instance.
        inputProducer: InputProducer class instance.
    """
    gnet.params['wd'] = 0.5
    gloss, sloss = gnet.loss(), snet.loss()
    loss = gloss  + sloss
    tf.scalar_summary('loss', loss)
    writer = tf.train.SummaryWriter(TB_SUMMARY, sess.graph)
    
    vars_train = gnet.variables + snet.variables

    # Backprop using SGD and updates vgg variables and sgnets variables
    global_step = tf.Variable(0, trainable=False)
    lr_exp = tf.train.exponential_decay(
            0.25, # Initial learning rate 
            global_step, 
            1000, # Decay steps 
            0.8, # Decay rate 
            name='sg_lr')

    tf.scalar_summary('Learning rate', lr_exp)
    optimizer = tf.train.GradientDescentOptimizer(lr_exp)
    train_op = optimizer.minimize(loss, var_list= vars_train, global_step=global_step)
    merged = tf.merge_all_summaries()

    sample_batches, target_batches = inputProducer.gen_batches(img, gt, n_samples=FLAGS.n_samples_per_batch, batch_sz=FLAGS.batch_size, pos_ratio=0.5, scale_factors=np.arange(0.5, 5., 0.2)) #np.array([1]))#
    print('Start training the SGNets........ for %s epoches'%FLAGS.iter_epoch_sg)
    saver = tf.train.Saver()
    step = 1
    loss_list = []
    for ep in range(FLAGS.iter_epoch_sg):
        print('Total batches in each epoch: ', len(sample_batches))
        for roi, target in zip(sample_batches, target_batches):
            #roi[roi>0] = 1 # neglect gaussian..set to 1 for target arear
            
            t = time.time()
            c4_maps, c5_maps = gen_sel_maps(sess, roi, vgg, idx_c4, idx_c5)
            
            fd = {gnet.input_maps: c5_maps, gnet.gt_M: target, 
                  snet.input_maps: c4_maps, snet.gt_M: target}
            
            # Initialization 
            if step == 1:
                loss_g = 10
                init_s = 0
                while loss_g > 1.5:
                    init_s += 1
                    sess.run(tf.initialize_variables(gnet.variables))
                    loss_g = sess.run(gloss, feed_dict=fd)
                    print('Initial Gnet Loss: ', loss_g, 'In steps: ', init_s)
                sess.run(tf.initialize_variables(snet.variables + [global_step]))
                
            
            pre_M_g, l, _, lr = sess.run([gnet.pre_M, loss, train_op, lr_exp], feed_dict=fd)
            
            loss_list += [l]
            if l <= 0.1:
                print('break learning!')
                break
            if step % 20 == 0:
                
                loss_ac = np.diff(np.diff(loss_list[-19:]))
                loss_ac_summary = tf.scalar_summary('Loss acceleration', loss_ac.mean())
                
                
                summary_img_g = tf.image_summary('pre_M', 
                                                 np.repeat(pre_M_g[...,np.newaxis], 3, axis=-1), name='GMap')

                summary, img_summary_g, ac_loss_summary = sess.run([merged, summary_img_g, loss_ac_summary], feed_dict=fd)

                writer.add_summary(summary, global_step=step)
                writer.add_summary(img_summary_g, global_step=step)
                writer.add_summary(ac_loss_summary, global_step=step)
                
                loss_std = np.std(loss_list[-19:])
                if loss_std <= 0.007:
                    
                    print('Stop learning! Last 10 batches Loss Std: ', loss_std)
                    break

            #if step % 20 == 0:
                print('Epoch: ', ep+1, 'Step: ', (ep+1)*step, 'Loss : %.2f'%l, \
                    'Speed: %.2f second/batch'%(time.time()-t), 'Lr: ', lr)
                #saver.save(sess, CKPT_MODEL)
            step += 1



print('Reading the first image...')
t_start = time.time()
## Instantiate inputProducer and retrive the first img
# with associated ground truth. 
inputProducer = InputProducer(IMG_PATH, GT_PATH)
img, gt, s  = next(inputProducer.gen_img)
roi_t0, _, rz_factor = inputProducer.extract_roi(img, gt)



ImportError: /home/makehave/anaconda3/bin/../lib/libgomp.so.1: version `GOMP_4.0' not found (required by /usr/lib/x86_64-linux-gnu/libsoxr.so.0)

In [ ]:
# Predicts the first img.
sess, vgg = init_vgg(roi_t0)
fd = {vgg.imgs: [roi_t0]}
gt_M = inputProducer.gen_mask((28,28)) # rank2 array


## At t=0. Train S and G Nets 
# Instainate SGNets with conv tensors and training.
# 1. feature maps selection
# 2. Train G and S networks.
idx_c4 = select_fms(sess, vgg.conv4_3_norm, gt, rz_factor, fd, FLAGS.sel_num)
idx_c5 = select_fms(sess, vgg.conv5_3_norm, gt, rz_factor, fd, FLAGS.sel_num)
snet = SNet('SNet', FLAGS.sel_num)
gnet = GNet('GNet', FLAGS.sel_num)


saver = tf.train.Saver()
saved_ckpt = os.path.join('checkpoint', FLAGS.model_name.split('_')[-1]+'.ckpt')
if os.path.exists(saved_ckpt):
    print('Found saved model %s, restoring! '%saved_ckpt)
    saver.restore(sess, saved_ckpt)
else:
    print('Not found saved model %s. Trainning! '%saved_ckpt)
    train_SGNets(sess, img, gt, vgg, snet, gnet, inputProducer, idx_c4, idx_c5)
    saver.save(sess, saved_ckpt)





In [ ]:
import skimage.io
import skimage.transform

show = skimage.io.imshow
%matplotlib inline

In [ ]:
import cv2
inputProducer = InputProducer(IMG_PATH, GT_PATH)
img, gt, s  = next(inputProducer.gen_img)
roi_t0, _, rz_factor = inputProducer.extract_roi(img, gt)

In [ ]:


inputProducer.roi_params['roi_scale'] = 1#0.0723 * 35
roi, _, rz_factor = inputProducer.extract_roi(img, gt)
print(gt)
show(roi)

In [ ]:
(66*55)

In [ ]:

inputProducer = InputProducer(IMG_PATH, GT_PATH)
img, gt, s  = next(inputProducer.gen_img)

gt

In [ ]:
img.astype(float).max()

In [ ]:
show(img)

In [ ]:
x,y,w,h = gt
roi_mean = img[y:y+h, x:x+w].mean()


In [ ]:
print(arear.max(), arear.min(), arear.mean())

In [ ]:
roi_mean

In [ ]:
roi_cur_mean

In [ ]:
np.ones([h, w], dtype=np.uint8)*int(roi_mean-roi_cur_mean)

In [ ]:
x,y,w,h = gt_last
roi_cur_mean = img[y:y+h, x:x+w].mean()
img[y:y+h, x:x+w] += int(roi_cur_mean- roi_mean)

In [ ]:
1.5/factor

In [ ]:
i = 255
(i>250 and i <260)

In [ ]:
## Instainate a tracker object, set apoproaite initial parameters.
tracker = TrackerContour()
inputProducer.roi_params['roi_scale'] = 2.5

print("Total time cost for initialization : %.2f s"%(time.time() - t_start))

# Iter imgs
inputProducer = InputProducer(IMG_PATH, GT_PATH)
img, gt, s  = next(inputProducer.gen_img)
x,y,w,h = gt
roi_mean = img[y:y+h, x:x+w].mean()

gt_last = gt

gt_list = []
pre_M_list = []
roi_list = []
res_list = []
diff_list = []

def dis2center(loc, sh=112):
    x,y,w,h = loc
    f1 = ((x-112)**2+(y-112)**2)**0.5
    f2 = ((x+w-112)**2+(y+w-112)**2)**0.5
    return f1, f2

def compute_score(loc_list, last_pre_loc, arear_list):
    scores_dis = []
    x0, y0, w0, h0 = last_pre_loc
    for loc in loc_list:
        x, y, w, h = loc
        #scores += [((x-x0)**2+ (y-y0)**2+ (w-w0)**2+ (h-h0)**2)**0.5]
        scores_dis += [abs(x-x0)+ abs(y-y0)+ abs(w-w0)+ abs(h-h0)]
    scores_dis = np.array(scores_dis) / max(scores_dis)
    arear_list = np.array(arear_list)/ max(arear_list)
    scores = scores_dis / arear_list
    best_idx = np.argmin(scores)
    return loc_list[best_idx], best_idx, scores[best_idx]

def compute_score1(loc_list, f1_0, f2_0):
    scores = []
    for loc in loc_list:
        f1, f2 = dis2center(loc)
        scores += [(f1-f1_0)**2+(f2-f2_0)**2]
    best_idx = np.argmin(scores)
    return loc_list[best_idx]


conf_scores = []
total_arear = 224**2
arear_in_bbox = []
for i in range(len(inputProducer.imgs_path_list)-1):


    t_enter = time.time()
    # Gnerates next frame infos
    img, gt_cur, s  = next(inputProducer.gen_img)
    
    # brighten img @inputProducer
    img = img.astype(np.float)
    x,y,w,h = gt_last
    roi_cur_mean = img[y:y+h, x:x+w].mean()
    img[y:y+h, x:x+w] += np.ones([h, w, 3])*int(roi_mean-roi_cur_mean)
    img[img>255] = 255
    
    if i%10==0:
        show(img)
        plt.show()
    
    #print(gt_last)
    ## Crop a rectangle ROI region centered at last target location.
    roi, _, rz_factor = inputProducer.extract_roi(img, gt_last)
    roi_list += [roi]
    
    
    # @inputproducer, remove low level pixel
    noise_value = 10#np.argmax(hist)*0.1
    roi[roi<noise_value] = roi.mean()

    ## Perform Target localiation predicted by GNet
    # Get heat map predicted by GNet
    c4_maps, c5_maps = gen_sel_maps(sess, roi, vgg, idx_c4, idx_c5)
    fd = {gnet.input_maps: c5_maps, snet.input_maps: c4_maps}

    pre_M_g, pre_M_s = sess.run([gnet.pre_M, snet.pre_M], feed_dict=fd)
    pre_M_list += [(pre_M_g.copy(), pre_M_s.copy())]
    
    
    pre_M = tracker.preporcess_heatmaps(pre_M_g, pre_M_s, resize=(224,224))
    
    tracker.predict_first_frame(pre_M_g, threshold=0.7)
        
    
    if i == 0:
        pre_M[pre_M<(0.7)] = 0
        
        cvuint8 = cv2.convertScaleAbs(pre_M)
        img2, contours, hierarchy = cv2.findContours(cvuint8, 1, 2)

        assert len(contours)>=1

        arear_list, bbox_list, COM_list = [], [], []
        for cnt in contours:

            x,y,w,h = cv2.boundingRect(cnt)
            bbox_list += [(x,y,w,h)]
            arear_list += [cv2.contourArea(cnt)]

            M = cv2.moments(cnt)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            COM_list += [(cx, cy)]
        best_idx = np.argmax(arear_list)
        pre_loc_roi = bbox_list[best_idx]
        com = COM_list[best_idx]
        last_arear = arear_list[best_idx]
        last_pre_loc_roi = pre_loc_roi
        conf_score = 1
        show(pre_M)
        plt.show()
    else:

        arear_list, bbox_list, COM_list, displace_list = [], [], [], []
        for shrehold in np.arange(0.3, 0.9, 0.05):
            pre_M = pre_M_g.copy()
            pre_M[pre_M<shrehold] = 0
            #show(pre_M)
            #plt.show()
            cvuint8 = cv2.convertScaleAbs(pre_M)
            img2, contours, hierarchy = cv2.findContours(cvuint8, 1, 2)
            if len(contours) < 1: continue

            for cnt in contours:
                try:
                    M = cv2.moments(cnt)
                    cx = int(M['m10']/M['m00'])
                    cy = int(M['m01']/M['m00'])
                except ZeroDivisionError:
                    continue
                COM_list += [(cx, cy)]
                x,y,w,h = cv2.boundingRect(cnt)
                if x+w >= 224 or y+h >224: continue
                if w/rz_factor>224 or h/rz_factor>224: continue
                bbox_list += [(x,y,w,h)]
                arear_list += [cv2.contourArea(cnt)]
        print('Total boxes: ', len(bbox_list), 'Current scale: ', inputProducer.roi_params['roi_scale'])
        inputProducer.roi_params['roi_scale'] = 1.5#(1.5/14) * len(bbox_list)
                
                #displace = ((cx-hs)**2 + (cy-hs)**2)**0.5
                #displace_list += [displace]
                #print(shrehold, 'shrehold', [x,y,w,h])

        #score_contour = (np.array(arear_list)- last_arear) * np.array(displace_list)
        #best_contour_idx = np.argmin(score_contour)
        #pre_loc_roi = bbox_list[best_contour_idx]
        #last_arear = arear_list[best_contour_idx]
        pre_loc_roi, idx, conf_score = compute_score(bbox_list, last_pre_loc_roi, arear_list)
        conf_scores += [conf_score]
        cx_pre, cy_pre = COM_list[idx]
        last_pre_loc_roi = pre_loc_roi
    
        
        
        
        #break
    ########
    
    if i % 10 ==0 or (i>250 and i <260):
        print('step: ', i)
        x,y,w,h = pre_loc_roi
        plt.figure(figsize=(10, 10)) 
        cv2.rectangle(pre_M_g,(x,y),(x+w,y+h),(1),2)
        plt.subplot(1,3,1), show(pre_M_g)
        plt.subplot(1,3,2), show(pre_M_s)
        plt.subplot(1,3,3), show(roi)
        plt.show()
        

        plt.plot(hist)

    


    
    #smoth_num = 
    if len(arear_in_bbox) > 50:
        #avg_arear = sum(arear_in_bbox[-15:])/15
        #roi_scale_coeff = (avg_arear/total_arear)*35
        #if roi_scale_coeff < 1: roi_scale_coeff = 0
        st, s0 = sum(arear_in_bbox[-50:-20])/30, sum(arear_in_bbox[-20:])/20
        try:
            factor = st-s0
        except ZeroDivisionError:
            factor = 1
        
        
        roi_scale_coeff = 1.5 # 0.001707333520651869* factor #- 0.004214667 * (factor) + 1.5
        inputProducer.roi_params['roi_scale'] = roi_scale_coeff
        print(roi_scale_coeff, arear_in_bbox[-30], arear_in_bbox[-1], factor, '$$$$$$$$$$$$$$$$$$$$$$$')
        #print('scale: ', inputProducer.roi_params['roi_scale'])
        #show(roi)
        #plt.show()
    
    

    print(pre_loc, 'best loc roi', gt_cur)
    
    diff = np.array(pre_loc) - np.array(gt_cur)
    if sum([abs(i) for i in diff]) > 50:
        diff_list += [i-1]
    #print('Step: ',i-1, 'pre - actual : ', diff, 'Conf level:', tracker.cur_best_conf)


    # Draw bbox on image. And print associated IoU score.
    x,y,w,h = [int(i) for i in pre_loc]
    xt, yt, wt, ht = gt_cur
    xl, yl, wl, hl = [int(i) for i in gt_last]
    imgcopy = img.copy().astype(np.uint8)
    if i!=0: imgcopy[cx_pre,cy_pre, 0] = 225
    cv2.rectangle(imgcopy,(x,y),(x+w,y+h),(225,0,0),2)
    cv2.rectangle(imgcopy,(xt,yt),(xt+wt,yt+ht),(0,225,0),1)

    #cv2.rectangle(imgcopy,(cx_pre-1,cy_pre-1),(cx_pre+1,cy_pre+1),(0,225,0),1)
    cv2.putText(imgcopy, 'conf score: %s'%conf_score,(5,20), font, 0.6,(255,0,0),1,cv2.LINE_AA)

    file_name = FLAGS.model_name + inputProducer.imgs_path_list[i-1].split('/')[-1]
    file_name = os.path.join(PRE_ROOT, file_name)
    
    x,y,w,h = pre_loc_roi
    test = cv2.rectangle(roi,(x,y),(x+w,y+h),(225,0,0),1)
    plt.imsave(file_name, imgcopy)

    font = cv2.FONT_HERSHEY_SIMPLEX
    


    #show(imgcopy)
    #plt.show()

    res_list += [imgcopy]

    gt_last = pre_loc
    gt_list += [pre_loc]


In [ ]:
factor

In [ ]:
- 0.004214667 * (factor) + 1.5

In [ ]:
1.5/factor

In [ ]:
-0.004214667041303738 * factor

In [ ]:
int(roi_mean-roi_cur_mean)

In [ ]:
a = np.ones([h, w, 3])*int(roi_mean-roi_cur_mean)

In [ ]:
show(roi)

In [ ]:
hist = np.bincount(roi.ravel(),minlength=256)

plt.plot(hist)

In [ ]:
show(roi)

In [ ]:
hist = np.bincount(roi.ravel(),minlength=256)

plt.plot(hist)

In [ ]:

vid_path_prefix = os.path.join(PRE_ROOT, FLAGS.model_name) 
os.system('ffmpeg -framerate 25 -i %s%%04d.jpg -c:v libx264 -profile:v high -crf 20 -pix_fmt yuv420p %s.mp4'\
          %(vid_path_prefix, FLAGS.model_name+'trail5'))

In [ ]:

show(imgcopy)

In [ ]:
imgcopy[cy_pre, cx_pre]

## try opencv adpaptie threlhoding and contouring 

In [ ]:
plt.subplot(1,2,1), show(pre_M_g)
plt.subplot(1,2,2), show(pre_M_g)


In [ ]:
k=411
show(res_list[k])
plt.show()
show(roi_list[k])
plt.show()
gm, sm = pre_M_list[k]
show(pre_M_list[k][0])
plt.show()
show(pre_M_list[k][1])

In [ ]:
show(gm)

In [ ]:
gm1 = gm.copy()
gm1[gm1<0.75]=0
show(gm1)

In [ ]:
cvuint8 = cv2.convertScaleAbs(gm1)
img2, contours, hierarchy = cv2.findContours(cvuint8, 1, 2)

assert len(contours)>=1

arear_list, bbox_list, COM_list = [], [], []
for cnt in contours:
    
    x,y,w,h = cv2.boundingRect(cnt)
    bbox_list += [(x,y,w,h)]
    arear_list += [cv2.contourArea(cnt)]
    
    M = cv2.moments(cnt)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    COM_list += [(cx, cy)]
best_idx = np.argmax(arear_list)
largest_contour = bbox_list[best_idx]
com = COM_list[best_idx]
print(largest_contour)
print(com)

In [ ]:
x,y,w,h = largest_contour
gm2 = cv2.rectangle(gm1,(x,y),(x+w,y+h),(1),1)
show(gm2)

In [ ]:
bbox_list

In [ ]:
#img = cv2.imread('star.jpg',0)
ret,thresh = cv2.threshold(gm,127,255,0)

In [ ]:
#gm = gm*225
gm = gm.astype(np.uint8)

In [ ]:
gray_image = cv2.cvtColor(gm, cv2.COLOR_BGR2GRAY)
cvuint8 = cv2.convertScaleAbs(gray_image)

In [ ]:
img2, contours, hierarchy = cv2.findContours(cvuint8, 1, 2)

cnt = contours[1]


In [ ]:
len(contours)

In [ ]:
x,y,w,h = cv2.boundingRect(cnt)
img = cv2.rectangle(gm,(x,y),(x+w,y+h),(0,255,0),1)

In [ ]:
print(x,y,w,h)

In [ ]:
show(img)

In [ ]:
vid_path_prefix = os.path.join(PRE_ROOT, FLAGS.model_name) 
os.system('ffmpeg -framerate 25 -i %s%%04d.jpg -c:v libx264 -profile:v high -crf 20 -pix_fmt yuv420p %s.mp4'\
          %(vid_path_prefix, FLAGS.model_name))

In [ ]:
len(inputProducer.imgs_path_list)

In [ ]:
for k,prem in enumerate(pre_M_list):
    if k % 20 ==0:
        show(prem)
        plt.show()
        show(roi_list[k])
        plt.show()

In [ ]:

t_enter = time.time()
# Gnerates next frame infos
img, gt_cur, s  = next(inputProducer.gen_img)

## Crop a rectangle ROI region centered at last target location.
roi, _, rz_factor = inputProducer.extract_roi(img, gt_last)

## Perform Target localiation predicted by GNet
# Get heat map predicted by GNet
c4_maps, c5_maps = gen_sel_maps(sess, roi, vgg, idx_c4, idx_c5)
fd = {gnet.input_maps: c5_maps, snet.input_maps: c4_maps}

pre_M_g, _ = sess.run([gnet.pre_M, snet.pre_M], feed_dict=fd)

#pre_M_g = imresize(pre_M_g, (224,224))

In [ ]:
os.system('ffmpeg -framerate 25 -i %s%%04d.jpg -c:v libx264 -profile:v high -crf 20 -pix_fmt yuv420p %sTEST.mp4'%(vid_path_prefix, FLAGS.model_name))

In [ ]:
call(['ffmpeg', '-framerate 25 -i %s%%04d.jpg -c:v libx264 -profile:v high -crf 20 -pix_fmt yuv420p %sTEST.mp4'%(vid_path_prefix, FLAGS.model_name)])

In [ ]:
show(pre_M_g)

In [ ]:
pre_M_g =  imresize(pre_M_g, (224,224))
# Localize target with monte carlo sampling.
pre_loc = tracker.predict_location(pre_M_g, gt_last, rz_factor, img)

gt_last = pre_loc
gt_list += [pre_loc]
print('pre: ', pre_loc, 'actual: ', gt_cur)

# Draw bbox on image. And print associated IoU score.
img_bbox = img_with_bbox(img, pre_loc,c=1)
show(img_bbox)

In [ ]:
show(img_bbox)

In [ ]:
pre_M_g_1 = pre_M_g.copy()

In [ ]:
pre_M_g_1[pre_M_g_1<0.5] = 0
show(pre_M_g_1)

In [ ]:
np.argmax(pre_M_g_1, axis=0)